### MNIST Digit Recognition

In [120]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import tensorflow as tf
import tensorflow.keras as K
import tensorflow_datasets as tfds
import time

#### Data Pre-processing

In [121]:
mnist_data, info=tfds.load(name='mnist', as_supervised=True, with_info=True)
mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

In [122]:
num_validation=tf.cast(0.1*info.splits['train'].num_examples, tf.int64)
num_test=tf.cast(info.splits['test'].num_examples, tf.int64)
num_train=tf.cast(info.splits['train'].num_examples, tf.int64)

In [123]:
def scale(image,label):
    image=tf.cast(image, tf.float32)
    image/=255
    return image, label
scaled_train_val= mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [124]:
BUFF_SIZE=10000
shuffled_train_val = scaled_train_val.shuffle(BUFF_SIZE)
mnist_val = shuffled_train_val.take(num_validation)
mnist_train= shuffled_train_val.skip(num_validation)

In [125]:
# Hyperparameters
batch_size=64

mnist_train = mnist_train.batch(batch_size)
mnist_val = mnist_val.batch(num_validation)
mnist_test=mnist_test.batch(num_test)

#To access the data in batches
val_in, val_target= next(iter(mnist_val))

#### Deep Learning Model prep

In [126]:
# (x_train, y_train), (x_test, y_test) = K.datasets.mnist.load_data(
#     path='mnist.npz'
# )

In [127]:
input_size=784
output_size=10
hidden_layer_size=64

model = K.Sequential([
                K.layers.Flatten(input_shape=(28,28,1)),
                K.layers.Dense(hidden_layer_size, activation='relu'),
                K.layers.Dense(hidden_layer_size+10, activation='relu'),
                K.layers.Dense(output_size, activation='softmax')    
])

In [128]:
optimizer=K.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [129]:
NUM_EPOCHS=9
model.fit(mnist_train, epochs=NUM_EPOCHS, validation_data=(val_in, val_target), validation_steps=64, verbose=2)

Epoch 1/9
844/844 - 12s - loss: 0.3283 - accuracy: 0.9071 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/9
844/844 - 11s - loss: 0.1470 - accuracy: 0.9566 - val_loss: 0.1275 - val_accuracy: 0.9615
Epoch 3/9
844/844 - 12s - loss: 0.1085 - accuracy: 0.9674 - val_loss: 0.1024 - val_accuracy: 0.9693
Epoch 4/9
844/844 - 11s - loss: 0.0849 - accuracy: 0.9742 - val_loss: 0.0878 - val_accuracy: 0.9743
Epoch 5/9
844/844 - 12s - loss: 0.0709 - accuracy: 0.9782 - val_loss: 0.0818 - val_accuracy: 0.9748
Epoch 6/9
844/844 - 11s - loss: 0.0605 - accuracy: 0.9810 - val_loss: 0.0659 - val_accuracy: 0.9792
Epoch 7/9
844/844 - 10s - loss: 0.0522 - accuracy: 0.9839 - val_loss: 0.0626 - val_accuracy: 0.9792
Epoch 8/9
844/844 - 11s - loss: 0.0448 - accuracy: 0.9861 - val_loss: 0.0640 - val_accuracy: 0.9818
Epoch 9/9
844/844 - 11s - loss: 0.0389 - accuracy: 0.9882 - val_loss: 0.0531 - val_accuracy: 0.9835


In [130]:
test_loss, test_acc= model.evaluate(mnist_test)

1/1 [==============================] 18.6062 - accuracy: 0.973 - 1s 1s/step - loss: 18.6062 - accuracy: 0.9730


In [131]:
print(test_loss, test_acc*100.)

18.606233596801758 97.29999899864197
